In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Burundi_Code_2017_penal.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
data

[Document(metadata={'producer': 'Acrobat Distiller 10.1.1 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2018-03-01T12:28:29+01:00', 'author': 'Droit Afrique', 'moddate': '2018-03-01T12:29:52+01:00', 'title': 'Burundi - Loi n°1/27 du 29 decembre 2017 portant revision du code penal (www.droit-afrique.com)', 'source': 'Burundi_Code_2017_penal.pdf', 'total_pages': 128, 'page': 0, 'page_label': '1'}, page_content='www.droit‐afrique.com\t Burundi\n\t \t\n\t\nCode\tpénal\t2017\t 1\nBurundi\t\nCode\tpénal\t\nLoi\tn°1/27\tdu\t29\tdécembre\t2017\t\n\t\n[NB\t‐\tLoi\tn°1/27\tdu\t29\tdécembre\t2017\tportant\trévision\tdu\tCode\tpénal]\t\nLivre\t1\t‐\tDes\tinfractions\tet\tde\tla\trépression\ten\tgénéral\t\nTitre\t1\t‐\tDes\tdispositions\tgénérales\t\nChapitre\t1\t‐\tDe\tl’infraction\ten\tgénéral\t\nArt.1.‐\tL’infraction\test\tune\taction\tou\tune\tomission\tqui\tse\tmanifeste\tc omme\tune\tatteinte\t\nà\tl’ordre\tsocial\tet\tque\tla\tloi\tsanctionne\tpar\tune\tpeine.\t\nArt.